In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import nltk
import re
import numpy as np
import string
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models


In [2]:
!python -m pip install spacy



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import spacy

c:\Users\koush\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
c:\Users\koush\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
c:\Users\koush\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords  
stop = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\koush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download("punkt")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\koush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\koush\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\koush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


In [8]:
import gensim
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
import gensim.corpora as cop
from pprint import pprint
from gensim.models import CoherenceModel
import pyLDAvis.gensim



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\koush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

In [10]:
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [56]:
df = pd.read_csv("vaccination_all_tweets.csv")

In [57]:
df_india = df['user_location'].tolist()

In [58]:
st = "India"
df_india_unique = set()
for i in df_india:
    if isinstance(i,str) and st in i:
        df_india_unique.add(i)

In [59]:
df_main = df[df["user_location"].isin(df_india_unique)]

In [60]:
df_main = df_main[df_main.source != "VaxBlr"] 

In [61]:
df_main = df_main.dropna()

In [62]:
df_main=df_main[['date','text']]

In [68]:
topic_data = df_main.query('date >="2021-01-16 00:00:00" and date <="2022-1-1 00:00:00"')

In [69]:
topic_data

,date,text
2947,2021-01-17 06:11:07,If we don't witness the same thing in India th...
3576,2021-01-16 05:47:45,#Norway has launched a detailed investigation ...
3577,2021-01-16 05:46:34,#horror #BreakingNews -\n\n23 people die in #N...
3586,2021-01-16 03:02:45,A global #CoronavirusVaccine rollout suffered ...
3712,2021-01-17 10:52:42,29 Dead in Norway after Getting Vaccinated.\n\...
...,...,...
227827,2021-11-23 11:23:54,News For #SputnikV vaccine takers:\n\nRussian ...
227879,2021-11-22 11:24:55,As Covishield and Covaxin push forward India's...
227931,2021-11-21 09:24:52,Anyone facing sudden changes in body after few...
228097,2021-11-17 10:13:16,#SputnikV vaccine has demonstrated 96.3% effic...


In [64]:
def clean_text(text):
    #Remove hyper links
    text = re.sub(r'https?:\/\/\S+', ' ', text)

    #Remove @mentions
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    
    #Removing hashtags
    text = re.sub(r'#[A-Za-z0-9]+', ' ', text)
    
    #Remove anything that isn't a letter, number, or one of the punctuation marks listed
    text = re.sub(r"[^A-Za-z0-9#'?!,.]+", ' ', text)   
    
    #The following regular expression are for removing apostrophes and expanding words
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    #text = re.sub(r'\b\w{1,4}\b', '', text)

    #Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    #remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    #Removing emojis
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)
    emoji_pattern.sub(r'', text)
    return text

In [70]:
topic_data['text'] = topic_data['text'].apply(clean_text)
topic_data['text'] = topic_data['text'].str.lower()

In [71]:
topic_data.shape

(19182, 2)

In [72]:
topic_data.head()

,date,text
2947,2021-01-17 06:11:07,if we do not witness the same thing in india t...
3576,2021-01-16 05:47:45,has launched a detailed investigation after ...
3577,2021-01-16 05:46:34,people die in after receiving officials doct...
3586,2021-01-16 03:02:45,a global rollout suffered a major blow friday ...
3712,2021-01-17 10:52:42,dead in norway after getting vaccinated note ...


In [24]:
def get_top_n_bigram(corpus, n=None):
  vec = CountVectorizer(stop_words = 'english', ngram_range=(2,3)).fit(corpus)
  bag_of_words = vec.transform(corpus)
  sum_words = bag_of_words.sum(axis=0) 
  words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
  words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
  return words_freq[:n]
common_words = get_top_n_bigram(topic_data['text'], 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])
bigram

,bigram,count
0,bharat biotech,982
1,emergency use,898
2,covid vaccine,667
3,second dose,430
4,use listing,241
5,emergency use listing,241
6,million doses,213
7,dr reddy,202
8,fully vaccinated,188
9,dose vaccine,180


In [25]:
def get_top_n_trigram(corpus, n=None):
  vec = CountVectorizer(stop_words = 'english', ngram_range=(3,3)).fit(corpus)
  bag_of_words = vec.transform(corpus)
  sum_words = bag_of_words.sum(axis=0) 
  words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
  words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
  return words_freq[:n]
common_words = get_top_n_trigram(topic_data['text'], 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])
trigram

,trigram,count
0,emergency use listing,241
1,emergency use authorisation,135
2,subject expert committee,110
3,emergency use india,101
4,approved emergency use,98
5,emergency use approval,96
6,bharat biotech covaxin,86
7,dashboard amp india,68
8,amp india figures,68
9,india figures cases,68


In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = topic_data.text.values.tolist()
topic_data = list(sent_to_words(data))

In [27]:
bigram = gensim.models.Phrases(topic_data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[topic_data], threshold=100)  


bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [28]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
# Remove Stop Words
data_words_nostops = remove_stopwords(topic_data)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])


['witness', 'thing', 'surely', 'lead', 'world']


In [30]:
topic_data = data_lemmatized

In [31]:
# Create Dictionary
id2word = cop.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [32]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [79]:
doc_topic_probs = [lda_model.get_document_topics(doc) for doc in corpus]

In [80]:
topic_freq = {}
for doc in doc_topic_probs:
    for topic, prob in doc:
        if topic not in topic_freq:
            topic_freq[topic] = 0
        topic_freq[topic] += 1

# print the resulting table
print(topic_freq)

{0: 18963, 1: 18964, 2: 18967, 3: 18952, 4: 18982, 5: 18979, 6: 18982, 7: 18968, 8: 18961, 9: 18985}


In [81]:
top_words=[]
for topic_id in range(lda_model.num_topics):
    topic_words = lda_model.show_topic(topic_id, topn=10)
    word_probs = [f'{word} ({prob:.2f})' for word, prob in topic_words]
    top_words.append(word_probs)

topic_table = pd.DataFrame(top_words, columns=[f'Topic {i+1}' for i in range(lda_model.num_topics)])

In [82]:
topic_table

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
0,trial (0.08),phase (0.06),efficacy (0.04),show (0.04),datum (0.04),fully (0.03),clinical (0.03),result (0.02),well (0.02),covaxin (0.02)
1,vaccinate (0.10),get (0.03),dose (0.02),day (0.02),state (0.02),people (0.02),go (0.02),govt (0.01),let (0.01),private_hospital (0.01)
2,approve (0.09),emergency (0.07),vaccine (0.05),approval (0.05),effective (0.03),covaxin (0.03),say (0.03),get (0.02),variant (0.02),child (0.02)
3,slot (0.03),work (0.03),vaccine (0.03),booster (0.02),safe (0.02),amp (0.01),test (0.01),tomorrow (0.01),sir (0.01),high (0.01)
4,take (0.08),vaccination (0.07),dose (0.07),today (0.04),vaccine (0.03),pm (0.03),second (0.02),people (0.02),receive (0.02),drive (0.01)
5,get (0.07),dose (0.06),production (0.04),finally (0.03),second (0.02),time (0.02),today (0.02),need (0.02),covaxin (0.02),covishield (0.01)
6,dose (0.09),vaccine (0.06),available (0.05),week (0.02),administer (0.02),sputnik (0.02),price (0.02),supply (0.01),import (0.01),say (0.01)
7,vaccine (0.13),covid (0.04),list (0.02),kid (0.01),new (0.01),manufacture (0.01),country (0.01),provide (0.01),launch (0.01),case (0.01)
8,amp (0.03),thank (0.03),approval (0.02),world (0.02),indian (0.02),submit (0.02),congratulation (0.02),permission (0.02),develop (0.01),add (0.01)
9,vaccine (0.04),good (0.03),news (0.02),government (0.02),soon (0.02),report (0.02),recommend (0.02),delay (0.01),base (0.01),study (0.01)


In [33]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"trial" + 0.056*"phase" + 0.043*"efficacy" + 0.039*"show" + '
  '0.037*"datum" + 0.031*"fully" + 0.026*"clinical" + 0.024*"result" + '
  '0.020*"well" + 0.020*"covaxin"'),
 (1,
  '0.102*"vaccinate" + 0.031*"get" + 0.025*"dose" + 0.021*"day" + '
  '0.017*"state" + 0.017*"people" + 0.016*"go" + 0.014*"govt" + 0.014*"let" + '
  '0.014*"private_hospital"'),
 (2,
  '0.095*"approve" + 0.067*"emergency" + 0.054*"vaccine" + 0.045*"approval" + '
  '0.035*"effective" + 0.028*"covaxin" + 0.028*"say" + 0.024*"get" + '
  '0.024*"variant" + 0.021*"child"'),
 (3,
  '0.033*"slot" + 0.028*"work" + 0.027*"vaccine" + 0.017*"booster" + '
  '0.017*"safe" + 0.014*"amp" + 0.014*"test" + 0.012*"tomorrow" + 0.012*"sir" '
  '+ 0.012*"high"'),
 (4,
  '0.080*"take" + 0.069*"vaccination" + 0.067*"dose" + 0.043*"today" + '
  '0.030*"vaccine" + 0.027*"pm" + 0.017*"second" + 0.017*"people" + '
  '0.017*"receive" + 0.013*"drive"'),
 (5,
  '0.073*"get" + 0.058*"dose" + 0.037*"production" + 0.027*"finally"

In [76]:
type(doc_lda)

gensim.interfaces.TransformedCorpus

In [35]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.34598128163956027


In [ ]:
pyLDAvis.gensim.prepare(lda_model, corpus,Dictionary(topic_data))

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.033378  0.061088       1        1  13.873993
7     -0.078520 -0.095083       2        1  12.361457
2     -0.126092  0.203273       3        1  11.016379
4      0.198266 -0.053865       4        1  10.628323
1      0.236970 -0.004043       5        1   9.411030
8     -0.110297  0.227384       6        1   9.258091
5      0.186227  0.044546       7        1   8.878712
0     -0.175088 -0.174676       8        1   8.772593
3     -0.046369 -0.149124       9        1   7.950244
9     -0.118476 -0.059499      10        1   7.849178, topic_info=          Term         Freq        Total Category  logprob  loglift
93     approve  1310.000000  1310.000000  Default  30.0000  30.0000
28   vaccinate   945.000000   945.000000  Default  29.0000  29.0000
112       dose  2527.000000  2527.000000  Default  28.0000  28.0000
13        take   873.000000   873.000000  Default  27.0000  27.0000
122   approval   818.000000   818.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
68         amp    63.569919   789.774043  Topic10  -4.7250   0.0252
36        come    43.061247   207.525907  Topic10  -5.1146   0.9721
22         say    55.607946   894.034524  Topic10  -4.8589  -0.2327
457       base    37.352868   138.368347  Topic10  -5.2568   1.2353
135      child    36.662232   373.358929  Topic10  -5.2754   0.2240

[579 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
434       4  0.233437       able
434      10  0.747000       able
867       7  0.960264     abroad
160       2  0.631724     accept
160       4  0.047144     accept
...     ...       ...        ...
393       4  0.002815       year
393       8  0.061930       year
393       9  0.520778       year
2455     10  0.980641   year_old
881       5  0.982013  yesterday

[1040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 3, 5, 2, 9, 6, 1, 4, 10])